# **ETL: Limpieza y preparación de los datos**

## **Líbrerias y módulos necesarios**

In [68]:
import warnings
import numpy as np
import pandas as pd
import sweetviz as sv
import plotly.express as px
warnings.filterwarnings('ignore')
from funciones import grafico_datos_faltantes

## **Conjunto de datos 1: Cáncer de mama**

```{note}
La información utilizada para este estudio está basada en el Certificado de Defunción del Ministerio de Salud de Colombia 
{cite}`minsalud_defuncion` . En este caso, observaciones sobre casos de defunción relacionados con el cáncer de mama. 

```

Inicialmente, se importa el primer conjunto de datos que contiene información detallada sobre la defunción de pacientes debido al cáncer de mama.

In [69]:
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/MAMA8923.csv"
data = pd.read_csv(url, delimiter=",", na_values=[" "], low_memory=False)

Veamos las primeras cinco observaciones para realizar un análisis inicial del conjunto de datos.

In [70]:
data.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,...,C_ANT22,C_ANT32,C_PAT2,IDPROFCER,CAUSA_667,P_PMAN_IRIS,CAUSA_MULT,TIPOFORMULARIO,Year,filter_$
0,50,1.0,1,1985,1,2,16,2,50.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
1,15,861.0,2,1985,1,2,24,1,15.0,861.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
2,50,573.0,1,1985,1,2,17,2,50.0,573.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
3,41,1.0,1,1985,1,2,14,2,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected
4,41,1.0,1,1985,1,2,22,2,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985,Selected


Las primeras cinco observaciones muestran datos con valores faltantes en varias columnas, diferencias en la codificación de variables y posibles inconsistencias en los identificadores de municipios e instituciones de salud. 

In [71]:
data.shape

(78481, 61)

El conjunto de datos inicial cuenta con **61** columnas (variables) y **78481** filas (observaciones).

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_DPTO        78481 non-null  int64  
 1   COD_MUNIC       78481 non-null  float64
 2   A_DEFUN         78481 non-null  int64  
 3   ANO             78481 non-null  int64  
 4   MES             78481 non-null  int64  
 5   SEXO            78481 non-null  int64  
 6   GRU_ED1         78481 non-null  int64  
 7   EST_CIVIL       78481 non-null  int64  
 8   CODPTORE        78439 non-null  float64
 9   CODMUNRE        78439 non-null  float64
 10  SIT_DEFUN       78481 non-null  int64  
 11  C_BAS1          78481 non-null  object 
 12  CONS_EXP        78481 non-null  int64  
 13  CAU_HOMOL       78481 non-null  int64  
 14  NOMBRE_ARCHIVO  78481 non-null  object 
 15  GRU_ED2         72563 non-null  float64
 16  AREA_RES        72477 non-null  object 
 17  perman_mun      5705 non-null  

A primera vista, observamos que las variables tienen nombres poco descriptivos, lo que dificulta la comprensión de su significado. Además, muchas de ellas están representadas como valores numéricos de tipo *float64*, aunque en realidad corresponden a categorías. Por ello, procederemos a redefinir los tipos de datos en función del significado de cada variable. También identificamos una alta presencia de datos faltantes en algunas columnas lo que requerirá un tratamiento adecuado más adelante.

In [73]:
column_types = {
    "COD_DPTO": "object", "COD_MUNIC": "object", "A_DEFUN": "object", "ANO": "object", "MES": "object", "SEXO": "object",
    "GRU_ED1": "object", "EST_CIVIL": "object", "CODPTORE": "object", "CODMUNRE": "object", "SIT_DEFUN": "object", "C_BAS1": "object",
    "CONS_EXP": "object", "CAU_HOMOL": "object", "AREA_RES": "object", "PMAN_MUER": "object", "COD_INST": "object", "NOM_INST": "object",
    "NIVEL_EDU": "object", "CODPRES": "object", "SEG_SOCIAL": "object", "MAN_MUER": "object", "CODOCUR": "object", "CODMUNOC": "object",
    "C_MUERTE": "object", "ASIS_MED": "object", "C_DIR1": "object", "C_ANT1": "object", "C_ANT2": "object", "C_ANT3": "object",
    "C_PAT1": "object", "C_MCM1": "object", "CAUSA_666": "object", "SIMUERTEPO": "object", "OCUPACION": "object", "IDPERTET": "object",
    "IDADMISALUD": "object", "IDCLASADMI": "object", "C_DIR12": "object", "C_ANT12": "object", "C_ANT22": "object", "C_ANT32": "object",
    "C_PAT2": "object", "IDPROFCER": "object", "Year": "object", "filter_$": "object"
}


In [74]:
data = pd.read_csv(url, delimiter=",", na_values=[" "], dtype = column_types)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_DPTO        78481 non-null  object 
 1   COD_MUNIC       78481 non-null  object 
 2   A_DEFUN         78481 non-null  object 
 3   ANO             78481 non-null  object 
 4   MES             78481 non-null  object 
 5   SEXO            78481 non-null  object 
 6   GRU_ED1         78481 non-null  object 
 7   EST_CIVIL       78481 non-null  object 
 8   CODPTORE        78439 non-null  object 
 9   CODMUNRE        78439 non-null  object 
 10  SIT_DEFUN       78481 non-null  object 
 11  C_BAS1          78481 non-null  object 
 12  CONS_EXP        78481 non-null  object 
 13  CAU_HOMOL       78481 non-null  object 
 14  NOMBRE_ARCHIVO  78481 non-null  object 
 15  GRU_ED2         72563 non-null  float64
 16  AREA_RES        72477 non-null  object 
 17  perman_mun      5705 non-null  

Nótese que el cambio se ha realizado exitosamente.

A continuación se eliminan columnas que no hacen parte del estudio.

In [75]:
data = data.drop(columns=[
    "NOMBRE_ARCHIVO",
    "GRU_ED2",
    "perman_mun",
    "tiem_per",
    "EST_CIVM",
    "OTRSITIODE",
    "HORA",
    "C_MUERTEB",
    "C_MUERTEC",
    "C_MUERTED",
    "C_MUERTEE",
    "CAUSA_667",
    "P_PMAN_IRIS",
    "CAUSA_MULT",
    "TIPOFORMULARIO"
])

### **Datos Faltantes**

Observemos a detalle los datos faltantes en este conjunto de datos.

In [76]:
missing_percent = (data.isnull().sum() / len(data)) * 100
missing_percent = missing_percent.sort_values(ascending=False)
missing_percent[missing_percent > 0]

CODMUNOC       95.974822
CODOCUR        95.974822
C_MCM1         95.871612
C_ANT32        95.572177
C_DIR12        95.293128
SIMUERTEPO     94.782177
C_ANT22        94.770709
C_ANT12        94.426677
C_PAT2         94.196047
MAN_MUER       91.963660
C_ANT3         84.475223
C_PAT1         83.720901
C_MUERTE       73.861189
IDCLASADMI     67.640575
COD_INST       63.969623
NOM_INST       63.765752
C_ANT2         62.891655
OCUPACION      52.438170
C_ANT1         47.900766
CAUSA_666      44.622265
C_DIR1         40.389394
IDADMISALUD    39.748474
PMAN_MUER      38.576216
IDPERTET       38.452619
IDPROFCER      38.452619
CODPRES        22.374842
ASIS_MED       16.488067
SEG_SOCIAL     16.488067
NIVEL_EDU      16.488067
AREA_RES        7.650259
CODPTORE        0.053516
CODMUNRE        0.053516
dtype: float64

In [77]:
missing_count = missing_percent[missing_percent > 0].count()
print("La cantidad de variables con datos faltantes es de: ", missing_count)
without_missing = missing_percent[missing_percent == 0].count()
print("La cantidad de variables sin datos faltantes es de: ", without_missing)
missing_above_70 = missing_percent[missing_percent > 70]
print("La cantidad de variables con más del 70% de datos faltantes es de: ", missing_above_70.count())
missing_below_70 = missing_percent[missing_percent <= 70]
print("La cantidad de variables con menos del 70% de datos faltantes es de: ", missing_below_70.count())


La cantidad de variables con datos faltantes es de:  32
La cantidad de variables sin datos faltantes es de:  14
La cantidad de variables con más del 70% de datos faltantes es de:  13
La cantidad de variables con menos del 70% de datos faltantes es de:  33


Veamos esto gráficamente:

In [78]:
grafico_datos_faltantes(data)

Al observar el gráfico, se puede decir que algunas variables presentan un alto porcentaje de datos faltantes, con varias de ellas superando el 90%, como `codcur` y `codmunoc`, que tienen un 95.97% de datos ausentes. Otras variables también muestran valores significativos de ausencia, como `SIMUERTEPO` (94.78%). Sin embargo, a medida que avanzamos en la gráfica, notamos que ciertas variables tienen menor cantidad de datos faltantes, con algunas en torno al 22%. Finalmente, hay un conjunto de variables sin valores faltantes (0%), lo que indica que esos datos están completamente disponibles en el conjunto analizado.

### **Eliminación de variables**


Para optimizar el análisis, eliminamos ciertas variables que no aportan valor significativo. La decisión se tomó con base en los siguientes criterios:  



**1. Variables eliminadas por recomendación de expertos**  
Los profesionales encargados del conjunto de datos recomendaron eliminar estas variables, ya que en este conjunto de datos NO son relevantes para el estudio y están representadas por otras variables:  
(`CAU_HOMOL`, `filter_$`)  



**2. Variables eliminadas por más del 75% de datos faltantes**  
Estas variables tienen un porcentaje muy alto de valores nulos, lo que dificulta su uso sin introducir sesgos en el análisis:  
**Observación:** Debido al alto porcentaje de datos faltantes, estas variables no pueden ser imputadas de manera confiable. Además, su eliminación no afecta el análisis, ya que no aportan información relevante o pueden derivarse de otras variables presentes en el conjunto de datos.

| Variable      | Porcentaje de Datos Faltantes |
|---------------|-------------------------------|
| `CODMUNOC`    | 95.97%                        |
| `CODOCUR`     | 95.97%                        |
| `C_MCM1`      | 95.87%                        |
| `C_ANT32`      | 95.57%                        |
| `C_DIR12`     | 95.29%                        |
| `SIMUERTEPO`  | 94.78%                        |
| `C_ANT22`     | 94.77%                        |
| `C_ANT12`     | 94.43%                        |
| `C_PAT2`      | 94.20%                        |
| `MAN_MUER`    | 91.96%                        |
| `C_ANT3`      | 84.48%                        |
| `C_PAT1`      | 83.72%                        |



**3. Variables que no son relevantes para el estudio**
Estas variables NO serán utilizadas en el enfoque de este estudio y, además, presentan una cantidad significativa de valores nulos:

| Variable     | Motivo de Eliminación                                                                                               | Porcentaje de Valores Nulos |
|--------------|----------------------------------------------------------------------------------------------------------------------|------------------------------|
| `A_DEFUN`    | No aporta información clínica ni contextual relevante para el análisis del cáncer de mama.                          | 0%                           |
| `ANO`        | Información redundante, ya que se cuenta con otra variable que representa el año.                                   | 0%                           |
| `AREA_RES`   | No se considera determinante para el enfoque actual del análisis; además, puede introducir sesgo geográfico.        | 7.65%                        |
| `C_ANT1`     | Causa antecedente 1. No aporta valor analítico específico para cáncer de mama y presenta alta cantidad de nulos.    | 47.9%                        |
| `C_ANT2`     | Redundante con otras variables sobre antecedentes; además, presenta un porcentaje elevado de datos faltantes.       | 62.89%                       |
| `C_DIR1`     | Causa directa. No se considera útil en el enfoque del estudio y tiene un número significativo de valores faltantes. | 40.39%                       |
| `COD_INST`   | Código de institución sin aplicación directa en este análisis y con muchos valores faltantes.                       | 63.97%                       |
| `CODPRES`    | Código sin utilidad práctica para el objetivo del estudio.                                                           | 22.37%                       |
| `CONS_EXP`   | Certificación médica o código administrativo sin relevancia clínica ni analítica para el modelo.                    | 0%                           |
| `IDCLASADMI` | Clasificación administrativa sin aporte directo al análisis y con alto número de valores faltantes.                  | 67.64%                       |
| `IDPERTET`   | Redundante con la variable `CODPTORE`, que ya representa la pertenencia étnica; además, muchos valores nulos.       | 38.45%                       |
| `IDPROFCER`  | Variable administrativa sin valor analítico para el objetivo del estudio.                                            | 38.45%                       |
| `NOM_INST`   | Información redundante con `COD_INST`, además no es imputable ni generalizable.                                     | 63.77%                       |
| `OCUPACION`  | Alto porcentaje de valores nulos; no se considera una variable relevante en la explicación de patrones de mortalidad por cáncer de mama. | 52.44%                        |


In [79]:
data = data.drop(columns=[
    # 1. Variables eliminadas por recomendación de expertos
    'CAU_HOMOL', 'filter_$',

    # 2. Variables eliminadas por más del 75% de datos faltantes
    'CODMUNOC', 'CODOCUR', 'C_MCM1', 'C_ANT32', 'C_DIR12',
    'SIMUERTEPO', 'C_ANT22', 'C_ANT12', 'C_PAT2', 'MAN_MUER',
    'C_ANT3', 'C_PAT1',

    # 3. Variables no relevantes para el estudio
    'A_DEFUN', 'ANO', 'AREA_RES', 'C_ANT2', 'COD_INST',
    'CODPRES', 'CONS_EXP', 'IDCLASADMI', 'IDPERTET', 'IDPROFCER',
    'NOM_INST', 'OCUPACION', 'C_ANT1', 'C_DIR1'
])

In [80]:
data.shape

(78481, 18)

Nótese que la eliminación se realizó de forma exitosa y ahora el conjunto de datos cuenta con solo **18** variables.

### **Renombrado de variables**

In [81]:
data = data.rename(columns={
    'COD_DPTO': 'depto_ocurr',       # Departamento donde ocurrió la defunción (DIVIPOLA)
    'COD_MUNIC': 'munic_ocurr',      # Municipio donde ocurrió la defunción (DIVIPOLA)
    'CODMUNRE': 'munic_res',        # Municipio de residencia del fallecido (DIVIPOLA)
    'MES': 'mes_def',                # Mes de la defunción
    'SEXO': 'sexo',                  # Sexo del fallecido
    'GRU_ED1': 'grupo_edad',         # Grupo de edad del fallecido
    'EST_CIVIL': 'estado_civil',     # Estado civil del fallecido
    'CODPTORE': 'depto_res',      # Departamento de residencia del fallecido (DIVIPOLA)
    'SIT_DEFUN': 'sitio_def',        # Sitio donde ocurrió la defunción
    'C_BAS1': 'causa_basica',        # Código de la causa básica de la defunción
    'PMAN_MUER': 'prob_muerte',      # Probable manera de muerte
    'NIVEL_EDU': 'nivel_edu',        # Nivel educativo del fallecido
    'SEG_SOCIAL': 'seg_social',      # Régimen de seguridad social
    'C_MUERTE': 'cert_medica',       # Cómo se determinó la causa de muerte
    'ASIS_MED': 'asistencia_med',    # Recibió asistencia médica antes de fallecer
    'CAUSA_666': 'causa_ops',        # Causa agrupada según la lista OPS 6/67
    'IDADMISALUD': 'ent_salud_cod',  # Código de la Entidad Administradora en Salud
    'Year': 'año_def'                # Año en que ocurrió la defunción
})


In [82]:
data.head()

,depto_ocurr,munic_ocurr,mes_def,sexo,grupo_edad,estado_civil,depto_res,munic_res,sitio_def,causa_basica,prob_muerte,nivel_edu,seg_social,cert_medica,asistencia_med,causa_ops,ent_salud_cod,año_def
0,50,001,01,2,16,2,50,001,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
1,15,861,01,2,24,1,15,861,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
2,50,573,01,2,17,2,50,573,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
3,41,001,01,2,14,2,41,001,1,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985
4,41,001,01,2,22,2,41,001,2,1749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1985


Ahora los nombres de cada variable son más legibles y ordenados. Lo que facilita el análisis de estos datos. [Diccionario de Variables](https://kmarcela11.github.io/ProyectoFinal_SeminarioInvestigativo/variables.html)

In [83]:
grafico_datos_faltantes(data)

### **Re-estructuración e imputación por variables**

En esta sección, se llevarán a cabo ajustes para mejorar la claridad y usabilidad de los datos. Primero, se renombrarán las variables con nombres más legibles y comprensibles, facilitando su interpretación. Además, las categorías etiquetadas como **"Sin información"** en algunas variables serán convertidas a **NaN**, lo que permitirá un análisis más estructurado, evitando sesgos y facilitando la imputación de datos en etapas posteriores. Estas transformaciones contribuirán a una mejor comprensión y optimización del conjunto de datos para su análisis.


#### **Variable: sexo**

Aunque el cáncer de mama también puede afectar a los hombres, en este caso únicamente se cuenta con datos de mujeres que han fallecido a causa de esta enfermedad.


In [84]:
data['sexo'].value_counts(dropna=False)

sexo
2    78481
Name: count, dtype: int64

In [85]:
data['sexo'] = data['sexo'].astype(str)
data['sexo'].replace('2', 'Femenino', inplace=True)
data['sexo'].value_counts(dropna=False)

sexo
Femenino    78481
Name: count, dtype: int64

#### **Variable: Mes de defunción**

En cuanto a la variable relacionada con el mes de defunción, no se cuenta con valores faltantes. En este caso, solo se recodifican las categorías para un mejor entendimiento de cada mes.

In [86]:
mes_dict_str = {
    "01": "Enero", "02": "Febrero", "03": "Marzo", "04": "Abril",
    "05": "Mayo", "06": "Junio", "07": "Julio", "08": "Agosto",
    "09": "Septiembre", "10": "Octubre", "11": "Noviembre", "12": "Diciembre"
}
data['mes_def'] = data['mes_def'].astype(str).str.zfill(2).replace(mes_dict_str)
data['mes_def'].value_counts(dropna=False)


mes_def
Diciembre     6782
Mayo          6765
Octubre       6724
Julio         6672
Agosto        6642
Junio         6540
Marzo         6493
Enero         6431
Septiembre    6426
Noviembre     6415
Abril         6379
Febrero       6212
Name: count, dtype: int64

#### **Variable: Estado civil**

Veamos que ocurre con la variable de estado civil.

In [87]:
data['estado_civil'].value_counts(dropna=False)
data['estado_civil'] = data['estado_civil'].astype(str)

In [88]:
data['estado_civil'] = data['estado_civil'].replace(['5', '6', '9'], np.nan)

estado_civil_dict = {
    "1": "Soltero",
    "2": "Casado",
    "3": "Viudo",
    "4": "Unión Libre, Divorciado/Otro"
}

data['estado_civil'] = data['estado_civil'].replace(estado_civil_dict)


In [89]:
data['estado_civil'].value_counts(dropna=False)

estado_civil
NaN                             23529
Casado                          17136
Soltero                         14058
Unión Libre, Divorciado/Otro    12878
Viudo                           10880
Name: count, dtype: int64

Se identificaron **23,529** datos faltantes. Dado que se trata de una variable difícil de imputar o estimar con precisión, se optó por asignar los valores faltantes en función de las proporciones observadas en las demás categorías, con el fin de no alterar la distribución general de la variable.


In [90]:
proporciones = data['estado_civil'].value_counts(normalize=True)
proporciones = proporciones[~proporciones.index.isna()]  # Excluir NaN si aparece
num_nan = data['estado_civil'].isna().sum()

In [91]:
valores_imputados = np.random.choice(
    proporciones.index,         # categorías (ej. Casado, Soltero, etc.)
    size=num_nan,               # cuántos valores imputar
    p=proporciones.values       # probabilidades según proporción
)

In [92]:
data.loc[data['estado_civil'].isna(), 'estado_civil'] = valores_imputados
data['estado_civil'].value_counts(dropna=False)

estado_civil
Casado                          24447
Soltero                         20104
Unión Libre, Divorciado/Otro    18371
Viudo                           15559
Name: count, dtype: int64

#### **Variable: Sitio de defunción**

In [93]:
data['sitio_def'].value_counts(dropna = False)

sitio_def
1    43365
2    27522
3     6844
4      695
6       41
5       13
9        1
Name: count, dtype: int64

In [94]:
data['sitio_def'] = data['sitio_def'].astype(str)
data['sitio_def'] = data['sitio_def'].replace( ['4','5', '6', '9'], np.nan)

sitio_def_dict = {"1": "Hospital o Clínica", "2": "Casa", "3": "Otro Sitio"}
data['sitio_def'] = data['sitio_def'].replace(sitio_def_dict)

data.loc[data['sitio_def'].isna(), 'sitio_def'] = "Otro Sitio"

In [95]:
data['sitio_def'].value_counts()

sitio_def
Hospital o Clínica    43365
Casa                  27522
Otro Sitio             7594
Name: count, dtype: int64

#### **Variable: Nivel educativo**

In [96]:
data['nivel_edu'] = data['nivel_edu'].replace(
    ['6', '7', '8', '9', '99', '10', '11', '12', '13'], pd.NA)

nivel_edu_dict = {
    "1": "Preescolar", "2": "Primaria", "3": "Secundaria",
    "4": "Superior", "5": "Ninguno"}

data['nivel_edu'] = data['nivel_edu'].replace(nivel_edu_dict)

data['nivel_edu'].fillna("No especificado", inplace=True)



In [97]:
data['nivel_edu'].value_counts(dropna = False)

nivel_edu
No especificado    25840
Primaria           25365
Secundaria         15637
Superior            7268
Ninguno             3838
Preescolar           533
Name: count, dtype: int64

En este caso, a los datos faltantes se les asocio la categoría **No especificado**, dado que no resulta coherente imputar estos datos de manera estadística.

#### **Variable: Causa agrupada con base en la Lista 6/67 de la OP**

In [98]:
data['causa_ops'].value_counts(dropna = False)

causa_ops
208      37004
NaN      35020
208.0     6457
Name: count, dtype: int64

In [99]:
data['causa_ops'] = data['causa_ops'].replace([208, '208.0', '208'], "Tumor Maligno")

data['causa_ops'].fillna("Tumor Maligno", inplace=True)

data['causa_ops'].value_counts(dropna=False)

causa_ops
Tumor Maligno    78481
Name: count, dtype: int64

In [100]:
data.drop('causa_ops', axis=1, inplace=True)

En este caso, al imputar los datos faltantes, se observó que, tratándose de defunciones por cáncer de mama, la causa agrupada corresponde en todos los casos a "tumor maligno de mama". Por lo tanto, todas las observaciones terminan teniendo la misma causa, lo que lleva a la decisión de eliminar esta variable por no aportar variabilidad al análisis.


#### **Variable: Asistencia médica**

In [101]:
data['asistencia_med'] = data['asistencia_med'].replace('9', '2')

asistencia_med_dict = {'1': "SI", '2': "NO", '3': "Ignorado"}

data['asistencia_med'] = data['asistencia_med'].map(asistencia_med_dict)

data['asistencia_med'].fillna('NO', inplace=True)
data['asistencia_med'].value_counts(dropna=False)


asistencia_med
SI          58251
NO          20136
Ignorado       94
Name: count, dtype: int64

#### **Variable: Probable manera de muerte**

In [102]:
data['prob_muerte'].value_counts(dropna=False)

prob_muerte
1      48200
NaN    30275
3          6
Name: count, dtype: int64

In [103]:
data.drop('prob_muerte', axis=1, inplace=True)

Se decide eliminar esta variable debido a la alta cantidad de datos faltantes (**30,275**), lo que representa una proporción significativa del total. Además, las categorías existentes están altamente desbalanceadas: la mayoría de los registros corresponden a una sola categoría, mientras que las demás tienen una representación mínima. En este contexto, no tendría sentido realizar un análisis sobre esta variable, ya que podría conducir a interpretaciones sesgadas o no representativas.


#### **Variable: Grupo de edad**

In [104]:
data['grupo_edad'] = data['grupo_edad'].astype(str)

data['grupo_edad'] = data['grupo_edad'].replace(['7', '8', '9', '26', '27', '28'], '25')


In [105]:
grupo_edad_dict = {
    "10": "15-19 años", "11": "20-24 años", "12": "25-29 años",
    "13": "30-34 años", "14": "35-39 años", "15": "40-44 años",
    "16": "45-49 años", "17": "50-54 años", "18": "55-59 años",
    "19": "60-64 años", "20": "65-69 años", "21": "70-74 años",
    "22": "75-79 años", "23": "80-84 años", "24": "85+ años",
    "25": "Edad desconocida"
}

data['grupo_edad'] = data['grupo_edad'].astype(str).replace(grupo_edad_dict)


In [106]:
data['grupo_edad'].value_counts(dropna = False)

grupo_edad
55-59 años          9373
60-64 años          9309
50-54 años          8708
65-69 años          8291
70-74 años          7473
45-49 años          6927
75-79 años          6151
80-84 años          4969
40-44 años          4880
85+ años            4554
Edad desconocida    3260
35-39 años          2778
30-34 años          1334
25-29 años           363
20-24 años            91
15-19 años            20
Name: count, dtype: int64

#### **Variable: Seguridad social**

In [107]:
data['seg_social'].replace('9', np.nan, inplace=True)

seg_social_dict = {
    "1": "Contributivo", "2": "Subsidiado", "3": "Vinculado",
    "4": "Particular", "5": "Otro", "6": "Ignorado"
}
data['seg_social'] = data['seg_social'].astype(str).replace(seg_social_dict)

data['seg_social'].replace('nan', np.nan, inplace=True)

data['seg_social'].fillna("Otro", inplace=True)

data['seg_social'].value_counts(dropna = False)

seg_social
Contributivo    33467
Subsidiado      24760
Otro            15287
Vinculado        3916
Particular        927
Ignorado          124
Name: count, dtype: int64

#### **Variable: cert_medica - ¿Cómo se determino la muerte?**

In [108]:
data['cert_medica'].value_counts(dropna = False)

cert_medica
NaN    57967
2      13906
"       3145
4       2439
9        769
1        214
3         41
Name: count, dtype: int64

In [109]:
data.drop('cert_medica', axis=1, inplace=True)

Se decide eliminar esta variable debido a la elevada cantidad de datos faltantes. Además, las categorías presentes son numerosas y están poco representadas, lo que dificulta una imputación adecuada y podría comprometer la calidad del análisis.


#### **Variable: Código de la entidad administradora en salud a la que pertenecía el fallecido**

In [110]:
data['ent_salud_cod'].value_counts(dropna = False)


ent_salud_cod
NaN    31195
1      24180
2      20012
5       2221
9        515
4        203
3        114
"         41
Name: count, dtype: int64

In [111]:
data['ent_salud_cod'].replace('9', np.nan, inplace = True)

In [112]:
ent_salud_dict = {
    "1": "EPS", "2": "EPS - Subsidiado", "3": "EAPB",
    "4": "ESE", "5": "EESS"}
data['ent_salud_cod'] = data['ent_salud_cod'].astype(str).replace(ent_salud_dict)

In [113]:
data.drop('ent_salud_cod', axis=1, inplace=True)

#### **Variable: Causa básica**

En esta variable se utilizan los códigos CIE-10 y CIE-9 para asociar las distintas causas de muerte relacionadas con la enfermedad del cáncer de mama. Estos códigos permiten identificar y clasificar de manera estandarizada las causas médicas registradas en los certificados de defunción.


In [114]:
data['causa_basica'].value_counts(dropna = False)

causa_basica
C509    66638
1749    11563
C500       49
C504       44
C501       33
C508       33
1740       30
C503       21
C502       15
1748       14
C506       13
C505       12
1741       10
1744        2
1745        1
1743        1
1742        1
1746        1
Name: count, dtype: int64

In [115]:
cie_dict = {
    'C500': 'Cuadrante superior interno',
    'C501': 'Cuadrante superior externo',
    'C502': 'Cuadrante inferior interno',
    'C503': 'Cuadrante inferior externo',
    'C504': 'Cuadrante superior interno',
    'C505': 'Región central',
    'C506': 'Prolongación axilar',
    'C508': 'Otras partes especificadas',
    'C509': 'Parte no especificada',
    '1740': 'Cuadrante superior interno',
    '1741': 'Cuadrante superior externo',
    '1742': 'Cuadrante inferior interno',
    '1743': 'Cuadrante inferior externo',
    '1744': 'Región central',
    '1745': 'Prolongación axilar',
    '1746': 'Otras partes especificadas',
    '1748': 'Múltiples localizaciones',
    '1749': 'Parte no especificada',
}

data['causa_basica'] = data['causa_basica'].map(cie_dict)


In [116]:
data['causa_basica'].value_counts(dropna = False)

causa_basica
Parte no especificada         78201
Cuadrante superior interno      123
Cuadrante superior externo       43
Otras partes especificadas       34
Cuadrante inferior externo       22
Cuadrante inferior interno       16
Múltiples localizaciones         14
Prolongación axilar              14
Región central                   14
Name: count, dtype: int64

## **Conjunto de datos 2: DIVIPOLA**


Este conjunto de datos proporciona los códigos oficiales utilizados para identificar departamentos, municipios y otras divisiones territoriales. Estos códigos sirven como referencia en el Conjunto de Datos 1 para estandarizar la información geográfica y facilitar su análisis.

In [117]:
column_types = {
    "Código Departamento": "object",
    "Código Municipio": "object",
    "Código Centro Poblado": "object",
    "Nombre Departamento": "object",
    "Nombre Municipio": "object",
    "Nombre Centro Poblado	": "object",
    "Tipo": "object"
}
url = "https://raw.githubusercontent.com/sePerezAlbor/Data/refs/heads/main/Divipola.csv"
data2 = pd.read_csv(url, delimiter = ",", na_values = [" "], dtype = column_types)
data2.head()

,Código Departamento,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo
0,05,05001,05001000,ANTIOQUIA,MEDELLÍN,MEDELLÍN,CM
1,05,05001,05001001,ANTIOQUIA,MEDELLÍN,PALMITAS,C
2,05,05001,05001004,ANTIOQUIA,MEDELLÍN,SANTA ELENA,C
3,05,05001,05001005,ANTIOQUIA,MEDELLÍN,PEDREGAL ALTO,IPM
4,05,05001,05001009,ANTIOQUIA,MEDELLÍN,ALTAVISTA,C


In [118]:
data2.shape

(9205, 7)

In [119]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Código Departamento    9205 non-null   object
 1   Código Municipio       9205 non-null   object
 2   Código Centro Poblado  9205 non-null   object
 3   Nombre Departamento    9205 non-null   object
 4   Nombre Municipio       9205 non-null   object
 5   Nombre Centro Poblado  9205 non-null   object
 6   Tipo                   9205 non-null   object
dtypes: object(7)
memory usage: 503.5+ KB


Nótese que se cuenta con un conjunto de datos bastante completo. 

### **Creación variable región**

Teniendo en cuenta los códigos del conjunto de datos inicial y los códigos proporcionados por el DANE en el conjunto de datos DIVIPOLA, se procede a clasificar los departamentos en sus respectivas regiones. Esto permite realizar análisis más globales a nivel nacional, facilitando la interpretación de los datos en un contexto geográfico más amplio.


In [120]:
condiciones = [
    data['depto_ocurr'].isin(['91', '18', '94', '95', '86', '97']),
    data['depto_ocurr'].isin(['5', '11', '15', '17', '25', '41', '54', '63', '66', '68', '73']),
    data['depto_ocurr'].isin(['8', '13', '20', '23', '44', '47', '70', '88']),
    data['depto_ocurr'].isin(['81', '85', '50', '99']),
    data['depto_ocurr'].isin(['19', '27', '52', '76'])
]

regiones = ['Amazonía', 'Andina', 'Caribe', 'Orinoquía', 'Pacífica']

data['region_Def'] = np.select(condiciones, regiones, default='Otra')


In [121]:
data['region_Def'].value_counts(dropna=False)

region_Def
Andina       48416
Caribe       14026
Pacífica     13947
Orinoquía     1582
Amazonía       510
Name: count, dtype: int64

## **Conjunto de datos final**

Tras analizar los dos conjuntos de datos previamente, procederemos a reemplazar los códigos de departamentos y municipios utilizando el conjunto de datos ``DIVIPOLA``. El objetivo de este proceso es mejorar la claridad y comprensión de la información, asignando a cada código su departamento y municipio correspondiente. Esto permitirá un análisis más preciso, estructurado y fácil de interpretar en el conjunto de datos general.  


Antes de asociar los dos conjuntos de datos, se realizó una corrección de errores de digitación y otros detalles presentes en las columnas de departamento y municipio. Esta limpieza previa fue necesaria para garantizar una correcta unión entre las bases y evitar inconsistencias durante el análisis.


In [122]:
data['depto_ocurr'] = data['depto_ocurr'].astype(str)
# data_prueba = data[data['Nombre_Departamento'].isna()]
# data_prueba['depto_ocurr'].value_counts(dropna=False)
data['depto_ocurr'].replace(['5'], '05', inplace=True)
data['depto_ocurr'].replace(['8'], '08', inplace=True)

In [123]:
def extraer_codigo_municipio(codigos_raw):
    codigos_municipio = []

    for codigo in codigos_raw:
        try:
            # Convertimos a número entero (quita ".0" si viene de float)
            codigo_int = int(float(codigo))
            # Nos quedamos con los últimos 3 dígitos, rellenando con ceros si hace falta
            ultimos_3 = str(codigo_int)[-3:].zfill(3)
            codigos_municipio.append(ultimos_3)
        except:
            continue  # Ignora valores no válidos

    return codigos_municipio

codigos_raw = data['munic_ocurr']
data['munic_ocurr'] = extraer_codigo_municipio(codigos_raw)

Se realizó la unión de los códigos de departamento y municipio, ya que en el primer conjunto de datos estos se encontraban por separado. Para poder hacer el cruce con el conjunto de datos DIVIPOLA, fue necesario construir el código completo, tal como se requiere para realizar la unión (merge) entre ambos conjuntos.


In [124]:
data["municipio"] = data["depto_ocurr"].astype(str) + data["munic_ocurr"].astype(str).str.zfill(3)
data['municipio_res'] = data['depto_res'].astype(str) + data['munic_res'].astype(str).str.zfill(3)

A continuación se realiza el mapeo de los códigos y la asignación de los nombres de departamentos y los municipios correspondientes a cada observación.

In [125]:
departamento_dict = dict(zip(data2['Código Departamento'], data2['Nombre Departamento']))
municipio_dict = dict(zip(data2['Código Municipio'], data2['Nombre Municipio']))
data['Nombre_Departamento_Def'] = data['depto_ocurr'].map(departamento_dict)
data['Nombre_Municipio_Def'] = data['municipio'].map(municipio_dict)
data['Nombre_Departamento_Res'] = data['depto_res'].map(departamento_dict)
data['Nombre_Municipio_Res'] = data['municipio_res'].map(municipio_dict)
data.head()


,depto_ocurr,munic_ocurr,mes_def,sexo,grupo_edad,estado_civil,depto_res,munic_res,sitio_def,causa_basica,...,seg_social,asistencia_med,año_def,region_Def,municipio,municipio_res,Nombre_Departamento_Def,Nombre_Municipio_Def,Nombre_Departamento_Res,Nombre_Municipio_Res
0,50,001,Enero,Femenino,45-49 años,Casado,50,001,Casa,Parte no especificada,...,Otro,NO,1985,Orinoquía,50001,50001,META,VILLAVICENCIO,META,VILLAVICENCIO
1,15,861,Enero,Femenino,85+ años,Soltero,15,861,Casa,Parte no especificada,...,Otro,NO,1985,Andina,15861,15861,BOYACÁ,VENTAQUEMADA,BOYACÁ,VENTAQUEMADA
2,50,573,Enero,Femenino,50-54 años,Casado,50,573,Casa,Parte no especificada,...,Otro,NO,1985,Orinoquía,50573,50573,META,PUERTO LÓPEZ,META,PUERTO LÓPEZ
3,41,001,Enero,Femenino,35-39 años,Casado,41,001,Hospital o Clínica,Parte no especificada,...,Otro,NO,1985,Andina,41001,41001,HUILA,NEIVA,HUILA,NEIVA
4,41,001,Enero,Femenino,75-79 años,Casado,41,001,Casa,Parte no especificada,...,Otro,NO,1985,Andina,41001,41001,HUILA,NEIVA,HUILA,NEIVA


In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78481 entries, 0 to 78480
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   depto_ocurr              78481 non-null  object
 1   munic_ocurr              78481 non-null  object
 2   mes_def                  78481 non-null  object
 3   sexo                     78481 non-null  object
 4   grupo_edad               78481 non-null  object
 5   estado_civil             78481 non-null  object
 6   depto_res                78439 non-null  object
 7   munic_res                78439 non-null  object
 8   sitio_def                78481 non-null  object
 9   causa_basica             78481 non-null  object
 10  nivel_edu                78481 non-null  object
 11  seg_social               78481 non-null  object
 12  asistencia_med           78481 non-null  object
 13  año_def                  78481 non-null  object
 14  region_Def               78481 non-nul

Vemos que en la columna de `Nombre_Municipio` hay dos datos faltantes. Veamos a que se debe esto.

In [127]:
data[data['Nombre_Municipio_Def'].isna()]

,depto_ocurr,munic_ocurr,mes_def,sexo,grupo_edad,estado_civil,depto_res,munic_res,sitio_def,causa_basica,...,seg_social,asistencia_med,año_def,region_Def,municipio,municipio_res,Nombre_Departamento_Def,Nombre_Municipio_Def,Nombre_Departamento_Res,Nombre_Municipio_Res
2285,99,496,Junio,Femenino,30-34 años,"Unión Libre, Divorciado/Otro",85,325,Hospital o Clínica,Parte no especificada,...,Otro,NO,1988,Orinoquía,99496,85325,VICHADA,NaN,CASANARE,SAN LUIS DE PALENQUE
3126,18,765,Abril,Femenino,35-39 años,Soltero,18,765,Casa,Parte no especificada,...,Otro,NO,1989,Amazonía,18765,18765,CAQUETÁ,NaN,CAQUETÁ,NaN


Al analizar las observaciones, se identificaron dos códigos de municipio que estaban mal digitados o eran erróneos. Dado que no es posible corregirlos con certeza y representan una proporción mínima del total, se decidió eliminar estas dos observaciones del conjunto de datos.


In [128]:
# Convertimos los códigos malos a una lista
codigos_erroneos = ['99496', '18765']

# Filtramos el DataFrame para excluirlos
data = data[~data['municipio'].isin(codigos_erroneos)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78479 entries, 0 to 78480
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   depto_ocurr              78479 non-null  object
 1   munic_ocurr              78479 non-null  object
 2   mes_def                  78479 non-null  object
 3   sexo                     78479 non-null  object
 4   grupo_edad               78479 non-null  object
 5   estado_civil             78479 non-null  object
 6   depto_res                78437 non-null  object
 7   munic_res                78437 non-null  object
 8   sitio_def                78479 non-null  object
 9   causa_basica             78479 non-null  object
 10  nivel_edu                78479 non-null  object
 11  seg_social               78479 non-null  object
 12  asistencia_med           78479 non-null  object
 13  año_def                  78479 non-null  object
 14  region_Def               78479 non-null  ob

Ahora si se cuenta con un conjunto de datos completo y sin datos nulos.

Vamos a organizar las columnas para una mejor representación visual.

In [138]:
columnas_ordenadas = [
    'region_Def', 'Nombre_Departamento_Def', 'Nombre_Municipio_Def', 'Nombre_Departamento_Res', 'Nombre_Municipio_Res', 'sitio_def', 'año_def', 
    'mes_def', 'sexo', 'estado_civil', 'grupo_edad', 'nivel_edu', 'seg_social',  'asistencia_med','municipio' ,    'municipio_res' ,           
  'causa_basica']
data_new = data[columnas_ordenadas]

In [139]:
data_new['Nombre_Departamento_Res'] = data_new['Nombre_Departamento_Res'].fillna(data_new['Nombre_Departamento_Def'])
capitales = {"AMAZONAS": "LETICIA","ANTIOQUIA": "MEDELLÍN","ARAUCA": "ARAUCA",  "ATLÁNTICO": "BARRANQUILLA", "BOLÍVAR": "CARTAGENA DE INDIAS",
"BOYACÁ": "TUNJA","CALDAS": "MANIZALES","CAQUETÁ": "FLORENCIA","CASANARE": "YOPAL", "CAUCA": "POPAYÁN","CESAR": "VALLEDUPAR", "CHOCÓ": "QUIBDÓ",
"CÓRDOBA": "MONTERÍA","CUNDINAMARCA": "BOGOTA DC", "GUAINÍA": "INÍRIDA", "BOGOTA DC":"BOGOTA DC", "GUAVIARE": "SAN JOSÉ DEL GUAVIARE", 
"HUILA": "NEIVA", "LA GUAJIRA": "RIOHACHA", "MAGDALENA": "SANTA MARTA","META": "VILLAVICENCIO", "NARIÑO": "PASTO","NORTE DE SANTANDER": "CÚCUTA", 
"PUTUMAYO": "MOCOA","QUINDIO": "ARMENIA","RISARALDA": "PEREIRA","SAN ANDRÉS": "SAN ANDRÉS","SANTANDER": "BUCARAMANGA","SUCRE": "SINCELEJO",
"TOLIMA": "IBAGUÉ","VALLE DEL CAUCA": "CALI", "VAUPÉS": "MITÚ","VICHADA": "PUERTO CARREÑO",
"ARCHIPIÉLAGO DE SAN ANDRÉS Y PROVIDENCIA Y SANTA CATALINA": "SAN ANDRÉS"
}
data_new['Nombre_Municipio_Res'] = data_new['Nombre_Municipio_Res'].fillna(data_new['Nombre_Departamento_Res'].map(capitales))

In [140]:
data_new.head()

,region_Def,Nombre_Departamento_Def,Nombre_Municipio_Def,Nombre_Departamento_Res,Nombre_Municipio_Res,sitio_def,año_def,mes_def,sexo,estado_civil,grupo_edad,nivel_edu,seg_social,asistencia_med,municipio,municipio_res,causa_basica
0,Orinoquía,META,VILLAVICENCIO,META,VILLAVICENCIO,Casa,1985,Enero,Femenino,Casado,45-49 años,No especificado,Otro,NO,50001,50001,Parte no especificada
1,Andina,BOYACÁ,VENTAQUEMADA,BOYACÁ,VENTAQUEMADA,Casa,1985,Enero,Femenino,Soltero,85+ años,No especificado,Otro,NO,15861,15861,Parte no especificada
2,Orinoquía,META,PUERTO LÓPEZ,META,PUERTO LÓPEZ,Casa,1985,Enero,Femenino,Casado,50-54 años,No especificado,Otro,NO,50573,50573,Parte no especificada
3,Andina,HUILA,NEIVA,HUILA,NEIVA,Hospital o Clínica,1985,Enero,Femenino,Casado,35-39 años,No especificado,Otro,NO,41001,41001,Parte no especificada
4,Andina,HUILA,NEIVA,HUILA,NEIVA,Casa,1985,Enero,Femenino,Casado,75-79 años,No especificado,Otro,NO,41001,41001,Parte no especificada


Finalmente, obtenemos un conjunto de datos más claro, comprensible y estructurado, lo que facilita su interpretación y lo deja listo para el análisis exploratorio.  


In [141]:
grafico_datos_faltantes(data_new)

El conjunto de datos evidentemente no tiene datos faltantes y esta listo para su análisis. 

In [142]:
data_new.shape

(78479, 17)

Finalmente, el conjunto de datos asociado a las defunciones relacionadas con cáncer de mama cuenta con **78479** observaciones y **15** variables.

In [143]:
data_new.to_excel('C:\\Users\\kamac\\OneDrive\\Desktop\\SeminarioInvestigativoUN\\data_to_eda.xlsx', index = False, engine = 'openpyxl')

```{note}

En el siguiente enlace se encuentra el diccionario de variables utilizado en este proyecto: [Diccionario de Variables](https://kmarcela11.github.io/ProyectoFinal_SeminarioInvestigativo/variables.html)
Este diccionario incluye una descripción detallada de cada una de las variables presentes en el conjunto de datos, facilitando su comprensión y el análisis del proyecto.

```

## **Referencias**
```{bibliography} references.bib
:style: plain
